# Головина Анна Анатольевна

In [158]:
from bs4 import BeautifulSoup
import collections
from fake_useragent import UserAgent
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import pandas as pd
from pymorphy2 import MorphAnalyzer
import random
import requests
from sklearn.metrics import accuracy_score
import time
from tqdm import tqdm

1. Сначала нам надо скачать данные -- соберите как минимум 60 (30 положительных и 30 отрицательных) отзывов на похожие продукты для составления "тонального словаря" и 10 отзывов для проверки качества.

Несмотря на все меры предосторожности, меня заблокировало 5 сайтов, поэтому код доступа оставила просто как достижение и в каждом цикле, связанном с краулером, использовала рандомные паузы в диапазоне от 5 до 20 секунд.

Выбрала сайт ["Право голоса"](https://pravogolosa.net), где можно отфильтровать страну и город отзыва, выбрать положительные, негативные или нейтральные отзывы (удобно для сохранения баланса при сборе данных) и настроить категорию товаров или услуг. И в целом это позволит изначально разделить тренировочные отзывы на положительные и отрицательные.

Оценивала тональность отзывов на IT-услуги в Москве, взяла 90 положительных и 90 отрицательных отзывов (от каждого потом отсекла по 5 для проверки качества).

In [181]:
session = requests.session()
response = session.get("https://pravogolosa.net")
response.status_code

200

Я пыталась написать функцию, которая позволит одинаково вытащить тексты отзывов как по фильтру "положительный", так и "негативный" без необходимости дублировать код. Но из-за размещения положительных и отрицательных отзывов на разных страницах, наличия форматированной строки в цикле (для перебора страниц), а также зацикленной дозаписи в список ссылок этого не вышло, вынесла в функции тот максимум, который смогла.

In [182]:
def html_extractor(link):
    """
    Общая функция для получения html страницы
    Аргумент - ссылка на страницу
    Вывод - html исходной страницы
    """
    user_agent = UserAgent().chrome
    response = requests.get(link, headers={'User-Agent':user_agent})
    page = response.text
    soup = BeautifulSoup(page, "html.parser")
    return soup

In [183]:
def previews_extractor(block_url):
    """
    Функция для получения html превью индивидуальных отзывов для одного блока
    Использует общую функцию парсинга
    Аргумент - ссылка на блок отзывов
    Вывод - список превью по одному блоку
    """
    block_soup = html_extractor(block_url)
    block_of_previews = block_soup.find_all(
        class_="ads_text_main"
    )  # превью отзывов по блоку (блок = 20 отзывов, дальше перелистывание)
    return block_of_previews

In [184]:
def review_text_extractor(review_url):
    """
    Функция для получения текста отзыва
    Использует общую функцию парсинга
    Аргумент - ссылка на отзыв
    Вывод - кортеж (заголовок отзыва, текст отзыва)
    """
    review_soup = html_extractor(review_url)

    review_text = review_soup.find("span", {"itemprop": "reviewBody"})
    if review_text.find("a"):
        review_text.a.decompose()
    review_text = review_text.text
    return review_text

Сначала соберем ссылки на полные тексты негативных отзывов.

In [185]:
neg_hrefs = []
for i in tqdm(range(20, 101, 20)):
    time.sleep(random.randint(5, 20))
    neg_url = f"https://pravogolosa.net/otzyvcategory?page=show_category&catid=173&order=0&ad_federalstate=%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D0%B0&ad_country=%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D1%8F&ad_tipre=%D0%9D%D0%B5%D0%B3%D0%B0%D1%82%D0%B8%D0%B2%D0%BD%D1%8B%D0%B9&start={i}"
    neg_previews = previews_extractor(neg_url)

    neg_hrefs += [
        f"https://pravogolosa.net{preview.find('h2').find('a').get('href')}"
        for preview in neg_previews
    ]

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:59<00:00, 11.97s/it]


После получения списка ссылок на негативные отзывы обработаем тексты и запишем их в список.

In [186]:
neg_texts = []

for href in tqdm(neg_hrefs):
    time.sleep(random.randint(5, 20))
    try:
        text = review_text_extractor(href)
        neg_texts.append(text)
    except:
        pass # пусть будет заглушка чтобы не генерить лишние файлы при запуске и не перегружать принтами

100%|██████████████████████████████████████████████████████████████████████████████████| 90/90 [20:41<00:00, 13.79s/it]


Теперь то же самое проделаем для положительных отзывов.

In [188]:
pos_hrefs = []
for i in tqdm(range(20, 101, 20)):
    time.sleep(random.randint(5, 20))
    pos_url = f"https://pravogolosa.net/otzyvcategory?page=show_category&catid=173&order=0&ad_federalstate=%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D0%B0&ad_country=%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D1%8F&ad_tipre=%D0%9F%D0%BE%D0%BB%D0%BE%D0%B6%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9&start={i}"
    pos_previews = previews_extractor(url)

    pos_hrefs += [
        f"https://pravogolosa.net{preview.find('h2').find('a').get('href')}"
        for preview in pos_previews
    ]

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:57<00:00, 11.56s/it]


In [189]:
pos_texts = []

for href in tqdm(pos_hrefs):
    time.sleep(random.randint(5, 20))
    try:
        text = review_text_extractor(href)
        pos_texts.append(text)
    except:
        pass

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [23:10<00:00, 13.90s/it]


Положительных отзывов заведомо больше, чем отрицательных, так что проверю размерность списка. Надеюсь, это не считается лишним принтом, я пыталась обосновать код, который мог бы возникнуть, если бы пришлось сокращать список положительных отзывов.

In [190]:
print(len(pos_texts))

90


2. Токенизируйте слова, приведите их к нижнему регистру и к начальной форме.

Для задачи, во-первых, подключим анализатор, а во-вторых, укажем стоп-слова, которые точно будут информационным мусором для тонального словаря.

In [224]:
morph = MorphAnalyzer()
stop_words = set(
    stopwords.words("russian")
)  # депрессия, в списке есть некоторые предлоги

Сделала функцию, в которой происходят и перевод в нижний регистр, и токенизация, и лемматизация.

In [225]:
def lemmatization(text):
    """
    Функция для получения лемм слов
    Аргумент - строка с текстом отзыва
    Вывод - список лемм
    """
    text = text.lower()
    tokenizer = RegexpTokenizer(r"[а-я0-9ё]+")
    text_tokenized = tokenizer.tokenize(text)
    lemmas = [
        morph.parse(word)[0].normal_form
        for word in text_tokenized
        if (word.isalpha() == True and word not in stop_words)
    ]
    return lemmas

Поскольку для составления словаря нужно 60+ отзывов, возьмем по 85 положительных и отрицательных (оставшиеся, как и написано в условии, пойдут на тестирование, их лемматизирую позже, в рамках другой функции).

In [226]:
neg_lemmas, pos_lemmas = [], []

for i in range(85): # обработаю по 85 первых положительных и отрицательных отзывов
    neg_lemmas += lemmatization(neg_texts[i])
    pos_lemmas += lemmatization(pos_texts[i])

3. Составьте 2 множества - в одном будут слова, которые встречаются только в положительных отзывах, а в другом - встречающиеся только в отрицательных. Попробуйте поиграть с частотностями и исключить шум.

Для поиска характерных для той или иной тональности слов составила функцию, которая сначала создает частотный словарь с фильтром, а затем собирает ключи словаря.

In [227]:
def keywords(list_of_lemmas):
    """
    Функция для получения наиболее частотных слов текста
    Аргумент - список лемм
    Вывод - список отфильтрованных по частотности лемм
    """
    c = collections.Counter()

    for lemma in list_of_lemmas:
        c[lemma] += 1
    # начиная с 6 (т.е. > 5) сильно проседают явно оценочные слова в положительных отзывах
    c_filtered = {i: c[i] for i in c.keys() if c[i] > 4}
    return list(c_filtered.keys())

Создаем temporary sets на основе результатов работы функции со списками лемм.

In [228]:
neg_temp_set = set(keywords(neg_lemmas))
pos_temp_set = set(keywords(pos_lemmas))

Чтобы получить наиболее точные результаты, выкидываем пересечение множеств (где скорее всего окажутся слова, связанные с IT-сферой как общим местом двух типов отзывов, случайно повторяющиеся слова и слова, имеющие двоякое значение в зависимости от контекста).

In [229]:
neg_set = neg_temp_set.difference(pos_temp_set)
pos_set = pos_temp_set.difference(neg_temp_set)

4. Создайте функцию, которая будет определять, положительный ли отзыв или отрицательный в зависимости от того, какие слова встретились в нём, и посчитайте качество при помощи accuracy.

Если честно, сама не смогла придумать ничего лучше чем это, но эта функция отвратительная и слишком примитивная, поэтому в итоге взяла семинарскую функцию.

In [230]:
def old_review_rating(review):
    """
    Функция для оценки тональности текста
    Аргумент - текст отзыва
    Вывод - оценка (1 = положительный, 0 = отрицательный, -1 = непонятно)
    """
    review_lemmas = lemmatization(review)
    neg_counter, pos_counter = 0, 0
    for lemma in review_lemmas:
        if lemma in pos_set:
            pos_counter += 1
        elif lemma in neg_set:
            neg_counter += 1
    if pos_counter > neg_counter:
        return 1
    elif pos_counter < neg_counter:
        return 0
    else:
        return -1

Вот модифицированная под задачу семинарская.

In [231]:
def review_rating(ratings, review):
    """
    Функция для оценки тональности текста
    Аргумент - текст отзыва
    Вывод - оценка
    """
    review_lemmas = lemmatization(review)

    rev_rating = collections.Counter()
    for grade, word_set in ratings.items():
        word_set = collections.Counter(word_set)
        for lemma in review_lemmas:
            rev_rating[grade] += int(word_set[lemma] > 0)
    return rev_rating.most_common()[0][0]

Создам словарь с двумя ключами-оценками, где 0 - отрицательная, а 1 - положительная оценка. Значениями в данном случае будут выступать множества (и в предыдущем задании, и здесь работаю с множествами во имя скорости обработки).

In [232]:
ratings_dict = {0: neg_set, 1: pos_set}

Реальная тональность отзывов мне заведомо известна благодаря выбранному сайту, и поскольку в список *test_data* отзывы добавлены вручную склейкой "хвостов" изначальных списков с отзывами, их корректную оценку также можно вручную записать через склейку списков нулей и единиц.

In [233]:
test_data = pos_texts[85:] + neg_texts[85:]

correct = [1 for i in range(len(pos_texts[85:]))] + [
    0 for i in range(len(neg_texts[85:]))
]
predicted = [review_rating(ratings_dict, item) for item in test_data]

In [234]:
accuracy_score(correct, predicted)

0.8

В целом, не самый плохой результат для выбранного подхода.

UPD: не очень понимаю, в чем дело, изначально я собирала 91 положительный отзыв и отсекала последний, точность была равна 0.7, на следующий день запустила код заново, собралось ровно 90 положительных отзывов, точность сначала взлетела до 1 (а я не очень верю, что подобный алгоритм действительно так хорошо предсказывает), затем я увеличила фильтр по частотности c (> 3) до (> 4), точность стала 0.8. Видимо, положительные отзывы дополнялись, поэтому первоначальные данные поехали.

**Итог**: реализация немного примитивная (старалась по максимуму отойти от семинарских материалов), но работает, на том спасибо.

5. Предложите как минимум 2 способа улучшить этот алгоритм определения тональности отзыва (1 балл за описание и реализацию каждого способа; если 2 способа описаны только текстом, это 1 балл. За третий и последующие способы дополнительных баллов не будет)

Все способы, какие смогла придумать, вдохновлены ридингом, то есть все решения про векторные представления слов.

**1 способ**: можно решить через bag-of-words (то есть нужно будет собрать все леммы всех отзывов в общий вектор и относительно словаря преобразовывать каждый отзыв в частотный вектор). Признаки (положительность/отрицательность тона) будут извлекаться из текста уже на основе частотности тех или иных слов в тексте каждого отзыва. В таком случае можно будет создать какую-нибудь модель (на основе логистической регрессии, k ближайших соседей или случайного леса) и обучить ее на классификацию по тональности.

**2 способ**: с помощью генерации эмбеддингов (то есть через word2vec). В таком случае в качестве инпута даем все тексты отзывов и получаем слова, преобразованные в вектора не на основе частотности, а на основе семантической близости (тогда слова с яркой оценочностью будут ближе друг к другу). Далее уже последует обучение модели, о котором сказано в предыдущем способе.